In [1]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops , stops
import os
import pickle
from opencc import OpenCC
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer , RobertaModel
from transformers import BertTokenizer , BertConfig , BertModel, XLNetTokenizer ,XLNetModel
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining , RobertaForSequenceClassification,BertForPreTraining
from torch.autograd import Variable
from transformers import BertForPreTraining
from torch import nn
import json
import requests

In [14]:

dataset = 2
path = './dataset/dataset' + str(dataset) + '/'

train_df = pd.read_csv(path + "tbrain_train.csv")
train_df = train_df.fillna('None')

test_df = pd.read_csv(path + "tbrain_test.csv")
test_df = test_df.fillna('None')

# train_df = pd.concat([train_df,test_df])
lm_path = './bert_wwm_pretrain_tbrain/'

# lm_path = './pretrain_roberta_on_TBdata'
print(train_df.shape)
print(test_df.shape)

(4426, 4)
(491, 4)


In [15]:

def clean_string(content):
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences , max_len):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < max_len:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = max_len
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li
    

In [4]:
def get_model_input_back512(df):
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    max_seq_length = 512
    train_x = []
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    train_y = []

    for index , row in df.iterrows():
        content = row['full_content']
        content = clean_string(content)

        content_ckip_names = ast.literal_eval(row['ckip_names'])
        name_ans = ast.literal_eval(row['name'])

        if len(name_ans) == 0:
            continue

        for ckip_name in content_ckip_names:
            content_max_length = 512-3-len(ckip_name)
            if len(content) >= content_max_length:
                content = content[-content_max_length:]

                    
            if ckip_name not in content:
                continue
            input_ids = tokenizer.encode(ckip_name, content)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            if ckip_name in name_ans:
                train_y.append(1)
            else:
                train_y.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            train_x.append((ckip_name,content)) 

    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    train_y = np.array(train_y)

    print(len(train_x))
    print(train_input_ids.shape)
    print(train_token_types.shape)
    print(train_attention_mask.shape)
    print(train_y.shape)
    
    return train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y

# train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y  = get_model_input_back512(train_df)
# test_x , test_input_ids ,test_token_types  , test_attention_mask , test_y  = get_model_input_back512(test_df)


In [5]:
def get_model_input_front512(df):
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    max_seq_length = 512
    train_x = []
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    train_y = []

    for index , row in df.iterrows():
        content = row['full_content']
        content = clean_string(content)

        content_ckip_names = ast.literal_eval(row['ckip_names'])
        name_ans = ast.literal_eval(row['name'])

        if len(name_ans) == 0:
            continue

        for ckip_name in content_ckip_names:
            content_max_length = 512-3-len(ckip_name)
            if len(content) >= content_max_length:
                content = content[:content_max_length]
                    
            if ckip_name not in content:
                continue
            input_ids = tokenizer.encode(ckip_name, content)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            if ckip_name in name_ans:
                train_y.append(1)
            else:
                train_y.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            train_x.append((ckip_name,content)) 

    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    train_y = np.array(train_y)

    print(len(train_x))
    print(train_input_ids.shape)
    print(train_token_types.shape)
    print(train_attention_mask.shape)
    print(train_y.shape)
    
    return train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y

# train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y  = get_model_input_front512(train_df)
# test_x , test_input_ids ,test_token_types  , test_attention_mask , test_y  = get_model_input_front512(test_df)


In [6]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [7]:
def get_model_input_split(df):
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    max_seq_length = 512
    train_x = []
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    train_y = []

    for index , row in df.iterrows():
        content = row['full_content']
        content = clean_string(content)

        content_ckip_names = ast.literal_eval(row['ckip_names'])
        name_ans = ast.literal_eval(row['name'])

        if len(name_ans) == 0:
            continue

        for ckip_name in content_ckip_names:
            content_max_length = 512-3-len(ckip_name)
            if len(content) >= content_max_length:
                split_content = cut_sent(content)
                chunks = combine_sentence(split_content , content_max_length)
                for chunk in chunks:
                    if ckip_name not in chunk:
                        continue
                    input_ids = tokenizer.encode(ckip_name, chunk)
                    if(len(input_ids)>512):
                        continue
                    sep_index = input_ids.index(tokenizer.sep_token_id)
                    num_seg_a = sep_index + 1
                    num_seg_b = len(input_ids) - num_seg_a
                    segment_ids = [0]*num_seg_a + [1]*num_seg_b

                    input_mask = [1] * len(input_ids)

                    while len(input_ids) < 512:
                        input_ids.append(0)
                        input_mask.append(0)
                        segment_ids.append(0)

                    if ckip_name in name_ans:
                        train_y.append(1)
                    else:
                        train_y.append(0)

                    train_input_ids.append(input_ids)
                    train_token_types.append(segment_ids)
                    train_attention_mask.append(input_mask)
                    train_x.append((ckip_name,chunk)) 


            else:

                input_ids = tokenizer.encode(ckip_name, content)
                if(len(input_ids)>512):
                    continue
                sep_index = input_ids.index(tokenizer.sep_token_id)
                num_seg_a = sep_index + 1
                num_seg_b = len(input_ids) - num_seg_a
                segment_ids = [0]*num_seg_a + [1]*num_seg_b

                input_mask = [1] * len(input_ids)

                while len(input_ids) < 512:
                    input_ids.append(0)
                    input_mask.append(0)
                    segment_ids.append(0)


                if ckip_name in name_ans:
                    train_y.append(1)
                else:
                    train_y.append(0)

                train_input_ids.append(input_ids)
                train_token_types.append(segment_ids)
                train_attention_mask.append(input_mask)
                train_x.append((ckip_name,content)) 



    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    train_y = np.array(train_y)

    print(len(train_x))
    print(train_input_ids.shape)
    print(train_token_types.shape)
    print(train_attention_mask.shape)
    print(train_y.shape)
    
    return train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y

# train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y  = get_model_input_split(train_df)
# test_x , test_input_ids ,test_token_types  , test_attention_mask , test_y  = get_model_input_split(test_df)


In [8]:
class TrainDataset(Dataset):
    def __init__(self, input_ids , token_type_ids , attention_mask , y ):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.y = y
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]

        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)
    

In [9]:
def get_eval(model,dataloader):
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for data in dataloader:
            tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors, 
                                labels=labels)
            pred = outputs[1]
            total += pred.size()[0]
            pred = torch.argmax(pred,dim=-1)
            correct += (pred==labels).sum().item()

    return correct/total 
    

In [ ]:
#  model training
BATCH_SIZE = 8
trainset = TrainDataset(train_input_ids ,train_token_types , train_attention_mask ,train_y)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE , shuffle=True)

testset = TrainDataset(test_input_ids ,test_token_types , test_attention_mask ,test_y)
testloader = DataLoader(testset, batch_size=BATCH_SIZE , shuffle=True)


from transformers import BertForSequenceClassification
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
print('dataset',dataset)

NUM_LABELS = 2
tokenizer = BertTokenizer.from_pretrained(lm_path)
model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
model = model.to(device)
# model.init_weights()
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 20 
for epoch in range(EPOCHS):
    running_loss = 0.0
    total = 0
    correct = 0
    for data in trainloader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors, 
                            labels=labels)
        loss = outputs[0]
        pred = outputs[1]
        total += pred.size()[0]
        pred = torch.argmax(pred,dim=-1)
        correct += (pred==labels).sum().item()

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    checkpoint_path = './QAModel/' + str(dataset) + '/' 
        
    torch.save(model.state_dict(),checkpoint_path + 'roberta_QA_split' + str(epoch) + '.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))
    print('train_acc:' ,correct/total)
    print('test_acc:' , get_eval(model,testloader))



In [10]:
def name_qa_binary_back(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
#     lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    

    for name in pred_name_list:
        if len(content) >= max_length:
            content = content[-max_length:]
            
        if name not in content:
            continue

        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])

        


In [11]:
def name_qa_binary_front(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
#     lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    

    for name in pred_name_list:
        if len(content) >= max_length:
            content = content[:max_length]
            
        if name not in content:
            continue

        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])

        


In [12]:
def name_qa_binary_split(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
#     lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content , max_length)
    
    for chunk in chunks:
        for name in pred_name_list:
            if len(chunk) >= max_length:
                print('error !!!! lenth > 500')
                continue
            if name not in chunk:
                continue

            input_ids = tokenizer.encode(name, chunk)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])

        


In [32]:
##### this is correct testing!!


dataset = 3
path = './dataset/dataset' + str(dataset) + '/'

train_df = pd.read_csv(path + "tbrain_train.csv")
train_df = train_df.fillna('None')

test_df = pd.read_csv(path + "tbrain_test.csv")
test_df = test_df.fillna('None')

# train_df = pd.concat([train_df,test_df])
lm_path = './bert_wwm_pretrain_tbrain/'
# lm_path = './pretrain_roberta_on_TBdata'
print(train_df.shape)
print(test_df.shape)


from transformers import BertForSequenceClassification
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
print('dataset',dataset)

base_path = './QAModel/' + str(dataset) + '/'

split_checkpoint = base_path + 'bert_wwm_QA_split1.pkl'
front_checkpoint = base_path + 'bert_wwm_QA_front0.pkl'
back_checkpoint = base_path + 'bert_wwm_QA_back3.pkl'

NUM_LABELS = 2
tokenizer = BertTokenizer.from_pretrained(lm_path)


# split model
split_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
split_model.load_state_dict(torch.load(split_checkpoint))
split_model = split_model.to(device)
split_model.eval()

split_pred = []
front_pred = []
back_pred = []
vote_ensemble_pred = []
front_end_or_pred = []
front_end_and_pred = []
# 先 or 在 and
front_end_or_split_and_pred = []
# 先 or 在 or
front_end_or_split_or_pred = []

ans = []

for index,row in test_df.iterrows():
    news = row['full_content']
    ckip_names = ast.literal_eval(row['ckip_names'])
    names = ast.literal_eval(row['name'])
    
    if len(names) == 0 :
        continue
    
    
    split_result = name_qa_binary_split(ckip_names , news ,split_checkpoint)
    split_result = list(set(split_result))
    
    front_result = name_qa_binary_front(ckip_names , news ,front_checkpoint)
    front_result = list(set(front_result))
    
    back_result = name_qa_binary_back(ckip_names , news ,back_checkpoint)
    back_result = list(set(back_result))
    
    
#     ensemble
    tmp = []
    all_result = split_result + front_result + back_result
    for name in list(set(all_result)):
        vote = 0
        vote += name in front_result
        vote += name in back_result
        vote += name in split_result
        if vote >= 2:
            tmp.append(name)
    vote_ensemble_pred.append(tmp)  
    
    tmp_and = []
    tmp_or = []
    all_result = front_result + back_result
    for name in list(set(all_result)):
        vote = 0
        vote += name in front_result
        vote += name in back_result
        if vote == 2: # and
            tmp_and.append(name)
        if vote >= 1: # or
            tmp_or.append(name)
            
    front_end_or_pred.append(tmp_or)  
    front_end_and_pred.append(tmp_and)  
    
    
    # 先 or 在 and
    front_end_or_split_and_pred.append(tmp_or and split_result)
    # 先 or 在 or
    front_end_or_split_or_pred.append(tmp_or or split_result)
    


    
    ans.append(names)
    split_pred.append(split_result)
    front_pred.append(front_result)
    back_pred.append(back_result)
    
print('dataset:',dataset)
# print('split:',eval_all(split_pred,ans))
# print('front:',eval_all(front_pred,ans))
# print('back:',eval_all(back_pred,ans))


print('vote_ensemble:',eval_all(vote_ensemble_pred,ans))
print('front_end_and_pred:',eval_all(front_end_and_pred,ans))
print('front_end_or_pred:',eval_all(front_end_or_pred,ans))
print('or_and :' , eval_all(front_end_or_split_and_pred,ans))
print('or_or :' , eval_all(front_end_or_split_or_pred,ans))



(4426, 4)
(491, 4)
device: cuda:0
dataset 3


Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining mod

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert_wwm_pretrain_tbrain/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dataset: 3
vote_ensemble: 0.8971685971685972
front_end_and_pred: 0.7459459459459461
front_end_or_pred: 0.9133848133848133
or_and : 0.9025740025740027
or_or : 0.9133848133848133


In [ ]:
# # ensemble

# from transformers import BertForSequenceClassification
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("device:", device)

# base_path = './QAModel/' + str(dataset) + '/'

# split_checkpoint = base_path + 'bert_wwm_QA_split1.pkl'
# front_checkpoint = base_path + 'bert_wwm_QA_front3.pkl'
# back_checkpoint = base_path + 'bert_wwm_QA_back4.pkl'

# NUM_LABELS = 2
# tokenizer = BertTokenizer.from_pretrained(lm_path)


# # split model
# split_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
# split_model.load_state_dict(torch.load(split_checkpoint))
# split_model = split_model.to(device)
# split_model.eval()


# # front model
# front_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
# front_model.load_state_dict(torch.load(front_checkpoint))
# front_model = front_model.to(device)
# front_model.eval()


# # back model
# back_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
# back_model.load_state_dict(torch.load(back_checkpoint))
# back_model = back_model.to(device)
# back_model.eval()


# # split_x , split_input_ids , split_token_types  , split_attention_mask , split_y  = get_model_input_back(test_df)
# # front_x , front_input_ids , front_token_types  , front_attention_mask , front_y  = get_model_input_front(test_df)
# # back_x , back_input_ids , back_token_types  , back_attention_mask , back_y  = get_model_input_back(test_df)

# front_x , front_input_ids , front_token_types  , front_attention_mask , front_y  = get_model_input_front(test_df)
# back_x , back_input_ids , back_token_types  , back_attention_mask , back_y  = get_model_input_back(test_df)



# BATCH_SIZE = 16
# frontset = TrainDataset(front_input_ids ,front_token_types , front_attention_mask ,front_y)
# frontloader = DataLoader(frontset, batch_size=BATCH_SIZE)

# backset = TrainDataset(back_input_ids ,back_token_types , back_attention_mask ,back_y)
# backloader = DataLoader(backset, batch_size=BATCH_SIZE)



# with torch.no_grad():
#     total = 0
#     correct = 0
#     for data in testloader:
#         tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
#         total += len(tokens_tensors)
        
        
        
#         split_outputs = split_model(input_ids=tokens_tensors, 
#                             token_type_ids=segments_tensors, 
#                             attention_mask=masks_tensors, 
#                             labels=labels)
#         split_pred = split_outputs[1]
#         split_pred = torch.argmax(split_pred,dim=-1)
        
        
#         back_outputs = back_model(input_ids=tokens_tensors, 
#                             token_type_ids=segments_tensors, 
#                             attention_mask=masks_tensors, 
#                             labels=labels)
#         back_pred = back_outputs[1]
#         back_pred = torch.argmax(back_pred,dim=-1)
        
#         front_outputs = front_model(input_ids=tokens_tensors, 
#                             token_type_ids=segments_tensors, 
#                             attention_mask=masks_tensors, 
#                             labels=labels)
#         front_pred = front_outputs[1]
#         front_pred = torch.argmax(front_pred,dim=-1)
        
        
#         vote_pred = split_pred.clone()
        
#         for i in range(len(tokens_tensors)):
#             vote = 0
#             vote += split_pred[i]
#             vote += back_pred[i]
#             vote += front_pred[i]
#             if vote >= 2:
#                 vote_pred[i] = 1
#             else:
#                 vote_pred[i] = 0
                  
#         correct += (vote_pred==labels).sum().item()

# print(dataset)
# print(correct/total)


In [ ]:
# # front end ensemble

# class EnsembleDataset(Dataset):
#     def __init__(self, input_ids , token_type_ids , attention_mask , y , x ):
#         self.input_ids = input_ids
#         self.token_type_ids = token_type_ids
#         self.attention_mask = attention_mask
#         self.y = y
#         self.x = x
#     def __getitem__(self,idx):
#         inputid = self.input_ids[idx]
#         tokentype = self.token_type_ids[idx]
#         attentionmask = self.attention_mask[idx]
#         y = self.y[idx]
#         x = self.x[idx]
#         return inputid , tokentype , attentionmask, y , x
#     def __len__(self):
#         return len(self.input_ids)


# from transformers import BertForSequenceClassification
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("device:", device)
# print(dataset)


# base_path = './QAModel/' + str(dataset) + '/'

# # split_checkpoint = base_path + 'bert_wwm_QA_split1.pkl'
# front_checkpoint = base_path + 'bert_wwm_QA_front4.pkl'
# back_checkpoint = base_path + 'bert_wwm_QA_back0.pkl'

# NUM_LABELS = 2
# tokenizer = BertTokenizer.from_pretrained(lm_path)


# # front model
# front_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
# front_model.load_state_dict(torch.load(front_checkpoint))
# front_model = front_model.to(device)
# front_model.eval()


# # back model
# back_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
# back_model.load_state_dict(torch.load(back_checkpoint))
# back_model = back_model.to(device)
# back_model.eval()



# front_x , front_input_ids , front_token_types  , front_attention_mask , front_y  = get_model_input_front512(test_df)
# back_x , back_input_ids , back_token_types  , back_attention_mask , back_y  = get_model_input_back512(test_df)



# BATCH_SIZE = 16
# frontset = EnsembleDataset(front_input_ids ,front_token_types , front_attention_mask ,front_y , front_x)
# frontloader = DataLoader(frontset, batch_size=BATCH_SIZE)

# backset = EnsembleDataset(back_input_ids ,back_token_types , back_attention_mask ,back_y , back_x)
# backloader = DataLoader(backset, batch_size=BATCH_SIZE)


# front_pred_names = []

# back_pred_names = []


# ckip_names = []

# all_ans_names = []

# with torch.no_grad():
#     total = 0
#     correct = 0
#     for data in frontloader:
#         tokens_tensors, segments_tensors, masks_tensors, labels  = [t.to(device) for t in data[:-1]]
        
#         text = data[-1]
#         names = np.array(text[0])
#         ckip_names = ckip_names + list(names)

        
            
        
#         outputs = front_model(input_ids=tokens_tensors, 
#                             token_type_ids=segments_tensors, 
#                             attention_mask=masks_tensors, 
#                             labels=labels)
#         pred = outputs[1]
#         pred = torch.argmax(pred,dim=-1)
#         pred = pred.cpu().detach().numpy()
#         pred_names = names[pred>0]
#         front_pred_names = front_pred_names + list(pred_names)

        
        
#         labels = labels.cpu().detach().numpy()
#         ans_names = names[labels==1]
#         all_ans_names = all_ans_names + list(ans_names)
        
        
# with torch.no_grad():
#     total = 0
#     correct = 0
#     for data in backloader:
#         tokens_tensors, segments_tensors, masks_tensors, labels  = [t.to(device) for t in data[:-1]]
        
#         text = data[-1]
#         names = np.array(text[0])
#         ckip_names = ckip_names + list(names)
        
            
        
#         outputs = back_model(input_ids=tokens_tensors, 
#                             token_type_ids=segments_tensors, 
#                             attention_mask=masks_tensors, 
#                             labels=labels)
#         pred = outputs[1]
#         pred = torch.argmax(pred,dim=-1)
#         pred = pred.cpu().detach().numpy()
#         pred_names = names[pred>0]
#         back_pred_names = back_pred_names + list(pred_names)
        
#         labels = labels.cpu().detach().numpy()
#         ans_names = names[labels==1]
#         all_ans_names = all_ans_names + list(ans_names)
        
# all_ans_names = list(set(all_ans_names))
# ckip_names = list(set(ckip_names))

# total = len(ckip_names)
# correct = 0

# total_pred = []
# total_ans  = []

# for ckip_name in ckip_names:    
#     back_pred_is_name = ckip_name in back_pred_names
#     front_pred_is_name = ckip_name in front_pred_names
    
#     vote_result = back_pred_is_name or front_pred_is_name
#     real_ans = ckip_name in all_ans_names
    
#     if real_ans == vote_result:
#         correct += 1
        

# print(dataset)
# print(correct/total)   

    


In [ ]:
#  這邊是小班的bio 直接跑就可以用

import pandas as pd
import ast
import numpy as np
import re
from zhon.hanzi import stops, non_stops
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertConfig, BertTokenizer, BertModel

import datetime #####
import math

PRETRAINED_MODEL_NAME = './bert_wwm_pretrain_tbrain/' # pretrained_bert_wwm
MODEL_PATH = './model_final_state_dict/pre_bert_wwm_bio_only_EPOCHES_19.pkl'

def clean_string_danny(content):
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
    result = []
    for i in range(math.ceil(len(content) / 511)):
        result.append(content[i*512 : i*512+511])
    return result

def bio_2_string(have_AML, BIO_tagging, ckip_result, origin_text, BIO_prob):
  result = []
  if (have_AML.item() == 0):
    result.append('')
  else:
#     for j in range(1, 512):
#       if (BIO_tagging[j] == 0):
#         start = j
#         end = j + 1
#         while (end < 512 and BIO_tagging[end] == 1):
#           end += 1
#         if (end > start + 1):
#           if (start <= 3):
#               s = origin_text[start-1 : end + 2] # -1 for CLS
# #               print(BIO_prob[start : end + 3])
#           else:
#               s = origin_text[start-1-1 : end + 2] # -1 for CLS
# #               print(BIO_prob[start-1 : end + 3])
# #           print('origin_span: ', origin_text[start-1 : end-1])
# #           print(s)
#           for k in range(len(ckip_result)):
#             if (len(ckip_result[k]) < 2):
#               continue
#             elif (re.findall(r"[%s]+" %non_stops, ckip_result[k]) != [] \
#                      or re.findall(r"[%s]+" %stops, ckip_result[k]) != []): # 有標點
#               continue
#             found = s.find(ckip_result[k])
#             if (found != -1):
# #               print('found: ', found)
#               result.append(ckip_result[k])
    full_str = ""
    for j in range(1, 512):
      if (BIO_tagging[j] == 0):
        start = j
        end = j + 1
        while (end < 512 and BIO_tagging[end] == 1):
          end += 1
        if (end > start + 1):
          if (start <= 3):
              s = origin_text[start-1 : end + 2] # -1 for CLS
#               print(BIO_prob[start : end + 3])
          else:
              s = origin_text[start-1-1 : end + 2] # -1 for CLS
#               print(BIO_prob[start-1 : end + 3])
#           print('origin_span: ', origin_text[start-1 : end-1])
#           print(s)
          full_str += s
    for k in range(len(ckip_result)):
      if (len(ckip_result[k]) < 2):
        continue
      elif (re.findall(r"[%s]+" %non_stops, ckip_result[k]) != [] \
                 or re.findall(r"[%s]+" %stops, ckip_result[k]) != []): # 有標點
        continue
      found = full_str.find(ckip_result[k])
      if (found != -1):
#       print('found: ', found)
        result.append(ckip_result[k])
      
    if (len(result) == 0):
      result.append('')
  return result

def get_predictions(model, tokens_tensors, segments_tensors, masks_tensors, ckip_names, origin_text, mode, binary):
  result = []
  with torch.no_grad():
      tokens_tensors = tokens_tensors.to("cuda:0")
      segments_tensors = segments_tensors.to("cuda:0")
      masks_tensors = masks_tensors.to("cuda:0")
      start = datetime.datetime.now().timestamp() ######
      outputs = model(input_ids=tokens_tensors, 
                  token_type_ids=segments_tensors, 
                  attention_mask=masks_tensors)
      end = datetime.datetime.now().timestamp()###########
#       print("through model time: ", end-start) ##########
      
      count = outputs[0].shape[0]
      for i in range(count):  # run batchsize times
        if (mode == 0):
            have_AML = outputs[0][i].argmax()
        else:
            have_AML = torch.tensor([binary])
        BIO_pred = outputs[0][i].argmax(1) # 3*512 into class label
        ckip_names_list = ast.literal_eval(ckip_names) # string to list
#         print(origin_text[i])
        start = datetime.datetime.now().timestamp() ######
        r = bio_2_string(have_AML, BIO_pred, ckip_names_list, origin_text[i], outputs[0][i])  #####
        end = datetime.datetime.now().timestamp()###########
#         print("bio_2_string time: ", end-start) ##########
        result.append(r)
  return result

""" model budling """
class AMLPredictModel(nn.Module):
    def __init__(self, config):
        super(AMLPredictModel, self).__init__()
        self.bert = BertModel.from_pretrained(PRETRAINED_MODEL_NAME, config = config)
        self.BIO_classifier = nn.Sequential(
                        nn.Linear(config.hidden_size, 3),
        ) # BIO tagging
        self.softmax = nn.Softmax(-1)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        BIO = self.BIO_classifier(outputs[0]) # 512*HIDDENSIZE word vectors
        BIO = self.softmax(BIO)
        
        outputs = (BIO,) + outputs[2:]
        return outputs
    

    
def get_AML_person(content, ckip_name, mode=0, binary=0):
    config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME, output_hidden_states=True)
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    model = AMLPredictModel(config)
    model.load_state_dict(torch.load(MODEL_PATH))
    start = datetime.datetime.now().timestamp() ######
    content = clean_string_danny(content)
    end = datetime.datetime.now().timestamp()###########
#     print("clean_string time: ", end-start) ##########
    start = datetime.datetime.now().timestamp() #####
    test_input_dict = tokenizer.batch_encode_plus(content, 
                          add_special_tokens=True,
                          max_length=512,
                          return_special_tokens_mask=True,
                          pad_to_max_length=True,
                          return_tensors='pt',
                          truncation=True)
    end = datetime.datetime.now().timestamp()###########
#     print("tokenizer time: ", end-start) ##########


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model = model.eval()
    r = get_predictions(model, test_input_dict['input_ids'], test_input_dict['token_type_ids'], test_input_dict['attention_mask'],\
                           ckip_name, content, mode, binary)
#     print(result)
    result = set()
    for i in range(len(r)):
        result = result | set(r[i])
    return result

In [ ]:

# 這邊是每天的 validation csv 輸出code

import pandas as pd

test_df = pd.read_csv('./tbrain/2020-07-29.csv')
validation_df =  pd.DataFrame(columns=['idx', 'article', 'BIO_output' , 'QAoutput', 'original_predict_name'])

count = 0
for index, row in test_df.iterrows():
    if(row['binary'] != 1):
        continue
    news = row['article']
    ckip_name = row['ckip_name']
    pred_name_list = ast.literal_eval(row['predict_name'])

#     這邊是call 小班的bio
    bio_output = get_AML_person(news, ckip_name, mode=1, binary=1)
    
#     小班的ｂｉｏ 再接我們的list
#     這邊就是call 我們的qa_ans
    qa_ans = check_pred_name_is_real_ans(bio_output , row['article'] , 4)
    qa_ans = list(set(qa_ans))
    print(qa_ans)
    validation_df.loc[count] = [str(index), news , str(set(bio_output)) , str(qa_ans) , row['predict_name']]
    count += 1

validation_df.to_csv('./tbrain/2020-07-29_QA_validation.csv',index=False)

In [ ]:

import pandas as pd

train_df = pd.read_csv('./dataset/tbrain_train (1).csv')
train_df = train_df.fillna('None')

test_df = pd.read_csv('./dataset/tbrain_test (1).csv')
test_df = test_df.fillna('None')

def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)


my_pred = []
ans = []
for index, row in test_df.iterrows():

    news = row['full_content']
    ckip_name = row['ckip_names']
    name = ast.literal_eval(row['name'])
    if len(name) == 0:
        continue
    
    bio_output = get_AML_person(news, ckip_name, mode=1, binary=1)
    
    qa_ans = check_pred_name_is_real_ans(bio_output , news, 1)
    
    qa_ans = list(set(qa_ans))
    print('-------')
    print('bio : ' , bio_output)
    print('pred : ' , qa_ans)
    print('ans : ' , name)
    my_pred.append(qa_ans)
    ans.append(name)

print(eval_all(my_pred,ans))
    
    

In [ ]:
def check_pred_name_is_real_ans_avg(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content , max_length)
    
    
    
    for chunk in chunks:
        for name in pred_name_list:
#             content_max_length = 512-3-len(name)
            if len(chunk) >= max_length:
                print('error !!!! lenth > 500')
                continue
            if name not in chunk:
                continue

            input_ids = tokenizer.encode(name, chunk)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            
            pred_name_list = np.array(name)
            
            val_dict = {}
            count_dict = {}
            for k in range(len(pred_name_list)):
                name = pred_name_list[k]
                probi = pred[k]
                if name in val_dict:
                    val_dict[name] += probi
                    count_dict[name] += 1
                else:
                    val_dict[name] = probi
                    count_dict[name] = 1
                    
            for name,count in count_dict.items():
                val_dict[name] /= count
            
            keys = list(val_dict.keys())
            values = list(val_dict.values())
            print(keys)
            print(values)
            ans = []
            
#             thresh hold
            th = 0.1
    
            for k in range(len(keys)):  
                if  values[k][1] > th:
                    ans.append(keys[k])
            return ans
        


In [ ]:
def check_pred_name_is_real_ans(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    def combine_sentence(sentences , max_len):
        li = []
        string = ''
        for k in range(len(sentences)):
            sentence = sentences[k]
            if len(string) + len(sentence) < max_len:
                string = string + sentence
            else:
                if string == '':
                    n = max_len
                    tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                    string = tmp_li.pop(-1)
                    li = li + tmp_li
                else:
                    li.append(string)
                    string = sentence
        if(string != ''):
            li.append(string)
        return li
    
    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content , max_length)
    
    
    
    for chunk in chunks:
        for name in pred_name_list:
#             content_max_length = 512-3-len(name)
            if len(chunk) >= max_length:
                print('error !!!! lenth > 500')
                continue
            if name not in chunk:
                continue

            input_ids = tokenizer.encode(name, chunk)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
#             print(pred, name)
            return list(pred_name_list[pred>0])
        


In [ ]:
def check_pred_name_is_real_ans_front512(pred_name_list,news,checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    
    
    for name in pred_name_list:
        content_max_length = 512-3-len(name)

        if len(content) >= content_max_length:
            content = content[:content_max_length]

        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])
        


In [ ]:
def check_pred_name_is_real_ans_back512(pred_name_list , news, checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    
    
    for name in pred_name_list:
        content_max_length = 512-3-len(name)

        if len(content) >= content_max_length:
            content = content[-content_max_length:]

        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])
        


In [ ]:
import pandas as pd


test_df = pd.read_csv('./tbrain/2020-07-30.csv')


validation_df =  pd.DataFrame(columns=['idx', 'article', 'BIO_output' , 'QAoutput', 'QAoutput_split','ckip+QAoutput','ckip+QAoutput_split'])

count = 0
for index, row in test_df.iterrows():
    if(row['binary'] != 1):
        continue
    news = row['article']
    ckip_name = ast.literal_eval(row['ckip_name'])
    pred_name_list = ast.literal_eval(row['predict_name'])
    bio_output = get_AML_person(news, row['ckip_name'], mode=1, binary=1)
    
    no_split_checkpint = './TB_multispan/Bert_wwm_ckip_name_is_ans_alldataset_epoch14.pkl'
    qa_ans_no_split = check_pred_name_is_real_ans_test_no_split(bio_output,news,no_split_checkpint)
    qa_ans_no_split = sorted(list(set(qa_ans_no_split)))
                             
                             
    qa_ans_no_split_without_bio = check_pred_name_is_real_ans_test_no_split(ckip_name,news,no_split_checkpint)
    qa_ans_no_split_without_bio = sorted(list(set(qa_ans_no_split_without_bio)))
    
                             
    qa_ans_split_without_bio = check_pred_name_is_real_ans(ckip_name , news, 4)
    qa_ans_split_without_bio = sorted(list(set(qa_ans_split_without_bio)))
    
    
    
    qa_ans_split = ast.literal_eval(row['predict_name'])
    qa_ans_split = sorted(list(set(qa_ans_split)))
    
    
    validation_df.loc[count] = [str(index), news , str(set(bio_output)) , str(qa_ans_no_split) , str(qa_ans_split),str(qa_ans_no_split_without_bio) , str(qa_ans_split_without_bio)]
    count += 1


validation_df.to_csv('./tbrain/2020-07-30_QA_validation.csv',index=False)